In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.447), (0.247, 0.243, 0.262))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
DenseNet = models.densenet121(pretrained = True)
DenseNet.classifier = nn.Linear(1024, 10)
DenseNet.to(device)
print(DenseNet)

/home/rushik/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rushik/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [5]:
# optimizer
optimizer = optim.SGD(DenseNet.classifier.parameters(), lr=0.001, momentum=0.9)
# loss function
criterion = nn.CrossEntropyLoss()

In [6]:
#transformations for metamorphic testing
transformations = [
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.ColorJitter(brightness=0.2),  # Slight brightness change
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Small translation
]

In [7]:
# Validation Function
def validate(model, test_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in tqdm(test_dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_running_loss += loss.item()
            _, preds = torch.max(output.data, 1)
            val_running_correct += (preds == target).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    val_loss = val_running_loss / len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct / len(test_dataloader.dataset)
    
    # Calculate metrics
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    return val_loss, val_accuracy, precision, recall, f1

In [8]:
def fit(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for  data, target in tqdm(train_dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

In [9]:
import torch
import torch.nn.functional as F

def weighted_vote_prediction(model, images, transformations, weights, device, num_classes=10):
    """
      weighted_preds: Tensor of shape [batch_size] with the weighted vote result.
      accepted_mask: Boolean tensor of shape [batch_size] indicating accepted samples.
    """
    model.eval()

    # 1) Original predictions
    with torch.no_grad():
        original_out = model(images)
        _, original_preds = torch.max(F.softmax(original_out, dim=1), dim=1)  # shape: [batch_size]

    # Initialize weighted vote sum using one-hot encoding of original predictions and its weight.
    weighted_sum = F.one_hot(original_preds, num_classes=num_classes).float() * weights[0]

    # 2) For each metamorphic transformation, get predictions and add weighted one-hot votes.
    for i, tf in enumerate(transformations):
        # Apply transformation on CPU and then move to device.
        x_tf = tf(images.cpu()).to(device)
        with torch.no_grad():
            out_tf = model(x_tf)
            _, preds_tf = torch.max(F.softmax(out_tf, dim=1), dim=1)
        # Convert predictions to one-hot and multiply by weight.
        weighted_sum += F.one_hot(preds_tf, num_classes=num_classes).float() * weights[i+1]

    # 3) Final weighted vote: for each sample, take the argmax of the weighted sum.
    weighted_preds = weighted_sum.argmax(dim=1)  # shape: [batch_size]

    # 4) Determine acceptance: accept if weighted vote equals the original prediction.
    accepted_mask = (weighted_preds == original_preds)

    return weighted_preds, accepted_mask


In [10]:
from tqdm import tqdm

def test_with_weighted_vote(model, loader, transformations, weights, device):
    """
    Evaluates the model on the test set 'loader' using weighted voting.
    For each batch:
      - Computes weighted vote predictions (using original + transformed outputs).
      - Accepts a sample only if the weighted vote equals the original prediction.
    
    Prints per-batch information and returns:
      - The accuracy among accepted samples.
      - The overall acceptance rate.
    """
    model.eval()
    total_samples = 0
    total_accepted = 0
    correct_accepted = 0

    for i, (images, labels) in enumerate(tqdm(loader)):
        images, labels = images.to(device), labels.to(device)

        # Get weighted vote predictions and acceptance mask for the batch.
        weighted_preds, accepted_mask = weighted_vote_prediction(model, images, transformations, weights, device)

        batch_size = labels.size(0)
        total_samples += batch_size
        batch_accepted = accepted_mask.sum().item()
        total_accepted += batch_accepted

        # Compute correct predictions among accepted samples.
        if batch_accepted > 0:
            batch_correct = (weighted_preds[accepted_mask] == labels[accepted_mask]).sum().item()
            correct_accepted += batch_correct

        print(f"Batch {i} | Accepted: {batch_accepted}/{batch_size}")

    overall_acceptance_rate = 100.0 * total_accepted / total_samples
    accepted_accuracy = 100.0 * correct_accepted / total_accepted if total_accepted > 0 else 0.0

    print(f"\nOverall Acceptance Rate: {overall_acceptance_rate:.2f}%")
    print(f"Accuracy among Accepted Samples: {accepted_accuracy:.2f}%")
    
    return accepted_accuracy, overall_acceptance_rate


In [11]:
weights = [0.5, 0.25, 0.25, 0.25]

acc_weighted, acceptance_rate = test_with_weighted_vote(DenseNet, testloader, transformations, weights, device)
print(f"Weighted Vote Accuracy among accepted samples: {acc_weighted:.2f}%")
print(f"Overall Acceptance Rate: {acceptance_rate:.2f}%")

  0%|▎                                                                                     | 2/625 [00:00<04:06,  2.53it/s]

Batch 0 | Accepted: 16/16
Batch 1 | Accepted: 15/16


  0%|▍                                                                                     | 3/625 [00:01<03:06,  3.33it/s]

Batch 2 | Accepted: 16/16


  1%|▋                                                                                     | 5/625 [00:01<02:26,  4.23it/s]

...

Batch 619 | Accepted: 15/16
Batch 620 | Accepted: 15/16


100%|███████████████████████████████████████████████████████████████████████████████████▋| 623/625 [01:47<00:00,  5.96it/s]

Batch 621 | Accepted: 16/16
Batch 622 | Accepted: 15/16


100%|████████████████████████████████████████████████████████████████████████████████████| 625/625 [01:47<00:00,  5.79it/s]

Batch 623 | Accepted: 16/16
Batch 624 | Accepted: 15/16

Overall Acceptance Rate: 96.34%
Accuracy among Accepted Samples: 8.32%
Weighted Vote Accuracy among accepted samples: 8.32%
Overall Acceptance Rate: 96.34%
